In [1]:
from openai_chat_agent.utils.google_search import GoogleSearch


In [2]:
query = "What is the typical life span of a cat?"
search = GoogleSearch()
summary, links = search(query)

print(f'I enquired:\n\"{query}\"\n')
print(f'I learned:\n{summary}\n')
print(f'My references:')
for link in links:
    print(f'{link["title"]}: {link["link"]}')

I enquired:
"What is the typical life span of a cat?"

I learned:
The average lifespan of domestic cats has increased over the years. In the 1980s, the average lifespan was seven years, which increased to nine years in 1995, and is now around 15 years in 2021. However, reliable information on cat lifespans is limited. Studies have estimated the mean lifespan to be between 13 and 20 years, with an average of around 15 years. Mixed-breed cats tend to have a longer lifespan compared to purebred cats, and breed-specific lifespans can vary. For example, Maine Coon cats have an average lifespan of 10-13 years, while Siamese cats can live to be 15-20 years old.

My references:
Aging in cats - Wikipedia: https://en.wikipedia.org/wiki/Aging_in_cats
How Long Do Cats Live? | PetMD: https://www.petmd.com/cat/care/how-long-do-cats-live
What Is the Average Lifespan of the Common Cat?: https://www.thesprucepets.com/lifespan-of-cats-552035


In [3]:
len(summary.split())

102